In [122]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid', rc={'figure.figsize':(20,8)})

# Reads csv files
companies = pd.read_csv('data/companies.tsv', sep='\t')
deals = pd.read_csv('data/deals.tsv', sep='\t')
sectors = pd.read_csv('data/sectors.tsv', sep='\t')
contacts = pd.read_csv('data/contacts.tsv', sep='\t')

# Removes spaces from column names
new_columns = dict(zip(contacts.columns, contacts.columns.str.strip()))
contacts.rename(new_columns, axis=1, inplace=True)

def first_output() -> pd.DataFrame:
    """Generates a dataframe that enables the creation and analysis of
    the sold value by month and by contact. Also creates a csv file in
    output directory

    Returns:
        pd.DataFrame: Dataframe containing contact name, deal value and month
    """

    contacts_deals = deals.merge(contacts, on='contactsId', how='left')
    # Creates a date column containing the 'month'/YY
    contacts_deals['monthYear'] = pd.to_datetime(contacts_deals['dealsDateCreated'])
    contacts_deals['monthYear'] = contacts_deals['monthYear'].dt.strftime('%b/%y')
    # Filters columns needed to create the sold value by month and by contact
    needed_columns = ['contactsName', 'dealsPrice', 'monthYear']
    first_output = contacts_deals[needed_columns]

    first_output.to_csv('output/first_output.csv', index=False)

    return first_output


if __name__ == '__main__':
    try:
        os.mkdir('output')
    except FileExistsError:
        pass
    first_output()


In [123]:
companies.head()

,companiesId,companiesName,companiesDateCreated,createdBy,companiesEmails,companiesPhones,employeesId,employeesName,usersResponsible,sectorKey
0,1,Class LLP,4/13/2019,johndoe,malesuada.vel@purus.edu,(56) 1742-4158,1,Damian Mathews,maryann,5
1,2,Vulputate Corporation,2/21/2019,johndoe,ut.erat.Sed@metusurna.net,(31) 8344-8808,2,Paul Leblanc,carlossuarez,3
2,3,Sed Dui Fusce Consulting,6/29/2017,johndoe,lacus.Aliquam@Proindolor.ca,(22) 6539-2114,3,Thomas Finléy,elisadoe,6
3,4,Orci Donec Nibh Incorporated,6/24/2017,johndoe,in@Sedeunibh.edu,(59) 9922-2903,4,Kelly Hamilton,joerobertson,3
4,5,Sagittis Nullam Vitae Ltd,8/6/2018,johndoe,pharetra@SuspendisseeleifendCras.org,(47) 2627-1232,5,Yoshio Prince,maryann,2


In [124]:
deals.head()

,dealsId,dealsDateCreated,dealsPrice,contactsId,companiesId
0,9,2/28/2018,2630,57,57
1,10,9/8/2018,8880,11,29
2,18,4/5/2018,3930,30,30
3,23,6/13/2018,2300,32,32
4,24,1/1/2018,3480,15,19


In [125]:
contacts.head()

,contactsId,contactsName,contactsDateCreated,contactsCreatedBy,contactsEmails,contactsPhones,contactsEmployers,employersId,contactsHomeAdress,contactsLatLong,contactsRelatedToLead,contactsResponsible
0,1,Damian Mathews,4/13/2019,johndoe,malesuada.vel@purus.edu,(56) 1742-4158,Class LLP,1,Ap #554-5077 Nunc Ave,"-29.55083, -95.44015",186,maryann
1,2,Paul Leblanc,2/21/2019,johndoe,ut.erat.Sed@metusurna.net,(31) 8344-8808,Vulputate Corporation,2,"P.O. Box 505, 3155 Dolor Av.","74.11207, 40.27304",86,carlossuarez
2,3,Thomas Finléy,6/29/2017,johndoe,lacus.Aliquam@Proindolor.ca,(22) 6539-2114,Sed Dui Fusce Consulting,3,209-1631 Velit. Ave,"79.42883, -110.04649",26,elisadoe
3,4,Kelly Hamilton,6/24/2017,johndoe,in@Sedeunibh.edu,(59) 9922-2903,Orci Donec Nibh Incorporated,4,"Ap #195-7212 Id, Rd.","-36.76197, 167.84722",61,joerobertson
4,5,Yoshio Prince,8/6/2018,johndoe,pharetra@SuspendisseeleifendCras.org,(47) 2627-1232,Sagittis Nullam Vitae Ltd,5,7919 Sit St.,"55.7959, 106.64754",52,maryann


In [126]:
sectors.head()

,sectorKey,sector
0,1,Bens de Consumo
1,2,Serviços
2,3,Tecnologia
3,4,Indústria
4,5,Varejo


In [131]:
companies_deals = deals.merge(companies[['companiesId', 'sectorKey']],
                                             on='companiesId', how='left')
sector_deals = companies_deals.merge(sectors, on='sectorKey', how='left')

sector_value = sector_deals.groupby('sector').sum().reset_index()

In [132]:
sector_value

,sector,dealsId,dealsPrice,contactsId,companiesId,sectorKey
0,Atacado,763,93270,690,693,96.0
1,Bens de Consumo,544,38840,392,392,8.0
2,Indústria,967,91070,564,638,76.0
3,Serviços,1085,99320,836,847,46.0
4,Tecnologia,743,87960,336,433,51.0
5,Varejo,876,48130,444,585,80.0


In [136]:
sector_percent = sector_value['dealsPrice'] / sector_value['dealsPrice'].sum()
sector_value['dealsPercent'] = sector_percent.round(3)

In [137]:
sector_value

,sector,dealsId,dealsPrice,contactsId,companiesId,sectorKey,dealsPercent
0,Atacado,763,93270,690,693,96.0,0.203
1,Bens de Consumo,544,38840,392,392,8.0,0.085
2,Indústria,967,91070,564,638,76.0,0.199
3,Serviços,1085,99320,836,847,46.0,0.217
4,Tecnologia,743,87960,336,433,51.0,0.192
5,Varejo,876,48130,444,585,80.0,0.105


In [135]:
sector_value['dealsPercent'].round(3)

0    0.203
1    0.085
2    0.199
3    0.217
4    0.192
5    0.105
Name: dealsPercent, dtype: float64

In [ ]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid', rc={'figure.figsize':(20,8)})

companies = pd.read_csv('data/companies.tsv', sep='\t')
deals = pd.read_csv('data/deals.tsv', sep='\t',
                    index_col='dealsDateCreated',
                    parse_dates=True)
sectors = pd.read_csv('data/sectors.tsv', sep='\t')
contacts = pd.read_csv('data/contacts.tsv', sep='\t')

# Removes spaces from column names
new_columns = dict(zip(contacts.columns, contacts.columns.str.strip()))
contacts.rename(new_columns, axis=1, inplace=True)

def value_per_month(df: pd.DataFrame, column: str):
    """Creates a plot with the total sold value by month and saves it
    in output directory

    Args:
        df: Time series dataframe
        column: Name of the numerical column to sum by month
    """

    value_by_month = df.resample('MS').sum()
    # Change the date format to 'mon'/YY
    value_by_month['month'] = value_by_month.index.to_series().dt.strftime('%b/%y')

    bar_plot = sns.barplot(x='month', y=column,
                           data=value_by_month, color='dodgerblue')

    # Set legend, title and x/y labels
    bar_plot.legend(['Sold Value'])
    bar_plot.set_title('Total Sold Value By Month', fontsize=20)
    bar_plot.set(xlabel='Month/Year',ylabel='Sold Value')

    # Add label to each bar
    for i, bar in enumerate(bar_plot.patches):
        h = bar.get_height()
        bar_plot.text(
            i, # bar index (x coordinate of text)
            h+1000, # y coordinate of text
            '{}'.format(int(h)),  # y label
            ha='center', 
            va='center', 
            size=14)

    # Save plot
    bar_plot.get_figure().savefig('output/sold_value_month.png')

    pass

def value_by_contact():
    """
    """

    new_columns = dict(zip(contacts.columns, contacts.columns.str.strip()))
    contacts.rename(new_columns, axis=1, inplace=True)


if __name__ == '__main__':
    try:
        os.mkdir('output')
    except FileExistsError:
        pass
    value_per_month(deals, 'dealsPrice')
